In [2]:
import findspark
findspark.init()
import pyspark
import os
import pandas as pd
import numpy as np 

sc = pyspark.SparkContext()
spark = pyspark.SQLContext(sc)

working_dir = os.getcwd()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 13:15:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/20 13:15:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
# Load parquet file into dataframe
df = spark.read.parquet("file://" + working_dir + "/testDataset/part-00000-711fabb0-5efc-4d83-afad-0e03a3156794.snappy.parquet")

In [3]:
from sqlalchemy import create_engine, text

engine = create_engine('postgresql://postgres@193.144.208.246:5432/tus_replica')

with engine.begin() as conn:
    query = text('select * from viajes_noviembre')
    df = pd.read_sql_query(query,conn)
    df = spark.createDataFrame(df) 

/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/andres/opt/anaconda3/envs/redes_neuronales_tus/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


# Exploration and data cleansing

#### Show total count of lines

In [4]:
# Count lines
df.count()

156291

#### Show the aspect of fiew records

In [7]:
# Show first 5 lines
df.show(5)

+------+--------+---------+-------------------+---------+------------+---------+------------+--------+----------+
|    id|nodo_org|nodo_dest|   instante_tickado|linea_org|sublinea_org|viaje_org|tipo_usuario| tarjeta| fecha_dia|
+------+--------+---------+-------------------+---------+------------+---------+------------+--------+----------+
|125679|      39|        1|2022-11-01 05:50:22|       41|           1|        3|           1|ES202954|2022-11-01|
|125680|      41|       45|2022-11-01 05:54:43|       41|           1|        3|          16|DP001458|2022-11-01|
|125681|     509|       40|2022-11-01 06:03:10|        3|           1|        1|           3|PS403368|2022-11-01|
|125682|     444|       60|2022-11-01 06:21:54|        3|           1|        1|           1|ES182742|2022-11-01|
|125683|     444|      355|2022-11-01 06:22:38|        3|           1|        2|           1|ES202471|2022-11-01|
+------+--------+---------+-------------------+---------+------------+---------+--------

#### Show all types of eventType

In [ ]:
df.createOrReplaceTempView("rawdata")
spark.sql("SELECT DISTINCT eventType FROM rawdata").show()

#### Remove rows with null or NaN values

In [5]:
# Remove rows with any null or NaN value
df.dropna
# Count lines
df.count()

156291

In [6]:
# Update created temp view for raw data
df.createOrReplaceTempView("rawdata")

### Remove unnecessary event types
Since we are going to predict estimation based on the start position, we won't need the rest of events (location, etc). Also, we remove rows with gps non valid values (0.0).

In [7]:
# 1. Create a view with only the values we need for processing
df = spark.sql("SELECT * FROM rawdata ORDER BY instante_tickado")
df.createOrReplaceTempView("cleands")
df.count()

156291

## Explore data transformation for a single device
The purpose is to reduce the problem so that we are sure that the transformations that we apply to the data produce the desired results. 
The aim of these transformations is to clean the dataset and take only the correctly paired start-trip / end-trip data.

In [8]:
# Show an example of data for a certain device
singleDevice = spark.sql("SELECT * FROM cleands WHERE tarjeta='ES182742'")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100, False)

+------+--------+---------+-------------------+---------+------------+---------+------------+--------+----------+
|id    |nodo_org|nodo_dest|instante_tickado   |linea_org|sublinea_org|viaje_org|tipo_usuario|tarjeta |fecha_dia |
+------+--------+---------+-------------------+---------+------------+---------+------------+--------+----------+
|125682|444     |60       |2022-11-01 06:21:54|3        |1           |1        |1           |ES182742|2022-11-01|
|127524|60      |511      |2022-11-01 17:38:34|3        |3           |3        |1           |ES182742|2022-11-01|
|127763|39      |494      |2022-11-06 06:31:27|3        |1           |2        |1           |ES182742|2022-11-06|
|99022 |304     |305      |2022-11-09 14:17:26|12       |1           |5        |1           |ES182742|2022-11-09|
|99038 |305     |309      |2022-11-09 14:19:16|12       |1           |6        |1           |ES182742|2022-11-09|
|137753|305     |301      |2022-11-17 13:53:26|12       |1           |6        |1       

### 1. Extract previous and next event for every row
To do this, we use LAG and LEAD window functions to shift the data. The result will add two new columns *previousEvent* and *nextEvent* to the dataset

In [ ]:
# Shift values of eventType to add previous and next event
singleDevice = spark.sql("SELECT *, LAG(eventType, 1) OVER (ORDER BY instante_tickado) AS previousEvent, \
LEAD(eventType, 1) OVER (ORDER BY instante_tickado) AS nextEvent FROM singledevice")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100)

### 2. Remove trip-start and trip-end non valid rows
The only valid data is that which defines a valid trip, that is, a trip-start followed by a trip-end event. Those trip-start with no trip-end after it, or those trip-end without a preceding trip-start are removed from the dataset.

In [ ]:
# Remove non valid trip-start and trip-end (those without its corresponding pair)
singleDevice = spark.sql("SELECT deviceId, eventTime, lat, lon, eventType FROM singledevice \
WHERE NOT ((eventType='trip-end' AND previousEvent='trip-end') \
OR (eventType='trip-start' AND nextEvent='trip-start'))")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(20, False)


### 3. Put all the data of every complete trip in the same row
We move the position and time data of every complete trip in a single row. This way we will be able to process each row as a trip, and we'll no longer need deviceId or eventType

In [ ]:
# Move start and end values of a trip to a single row. We won't need deviceId any more
singleDevice = spark.sql("SELECT eventTime AS eventTimeStart, \
LEAD(eventTime, 1) OVER (ORDER BY eventTime) AS eventTimeEnd, lat AS latStart, lon AS lonStart, \
LEAD(lat, 1) OVER (ORDER BY eventTime) AS latEnd, \
LEAD(lon, 1) OVER (ORDER BY eventTime) AS lonEnd, \
eventType FROM singledevice")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice = spark.sql("SELECT eventTimeStart, eventTimeEnd, latStart, lonStart, latEnd, lonEnd \
FROM singledevice WHERE eventType='trip-start'")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100, False)

## Apply the transformations to the whole dataset

### 1. Extract previous and next event for every row
To do this, we use LAG and LEAD window functions to shift the data. The result will add two new columns *previousEvent* and *nextEvent* to the dataset

In [ ]:
# Shift values of eventType to add previous and next event
allDevices = spark.sql("SELECT *, LAG(eventType, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS previousEvent, \
LEAD(eventType, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS nextEvent FROM cleands")
allDevices.createOrReplaceTempView("alldevices")
allDevices.show(20)

### 2. Remove trip-start and trip-end non valid rows
The only valid data is that which defines a valid trip, that is, a trip-start followed by a trip-end event. Those trip-start with no trip-end after it, or those trip-end without a preceeding trip-start are removed from the dataset.

In [ ]:
# Remove non valid trip-start and trip-end (those without its corresponding pair)
allDevices = spark.sql("SELECT deviceId, eventTime, lat, lon, eventType FROM alldevices \
WHERE NOT ((eventType='trip-end' AND previousEvent='trip-end') \
OR (eventType='trip-start' AND nextEvent='trip-start'))")
allDevices.createOrReplaceTempView("alldevices")
allDevices.show(20, False)

In [ ]:
allDevices.count()

### 3. Put all the data of every complete trip in the same row
We move the position and time data of every complete trip in a single row. This way we will be able to process each row as a trip, and we'll no longer need deviceId or eventType

In [ ]:
# Move start and end values of a trip to a single row. We won't need deviceId any more
allDevices = spark.sql("SELECT eventTime AS eventTimeStart, \
LEAD(eventTime, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS eventTimeEnd, lat AS latStart, lon AS lonStart, \
LEAD(lat, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS latEnd, \
LEAD(lon, 1) OVER (PARTITION BY deviceId ORDER BY eventTime) AS lonEnd, \
eventType FROM alldevices")
allDevices.createOrReplaceTempView("alldevices")
allDevices = spark.sql("SELECT eventTimeStart, eventTimeEnd, latStart, lonStart, latEnd, lonEnd \
FROM alldevices WHERE eventType='trip-start'")
allDevices.createOrReplaceTempView("alldevices")
allDevices.show(20, False)

In [ ]:
allDevices.count()

### Remove duplicates
After examining the resulting data, we observe that there many registers with exactly the same data for the time start and de starting and end position, so we remove the redundant data to avoid bias on the predicting model

In [ ]:
allDevices = allDevices.dropDuplicates(['eventTimeStart', 'latStart', 'lonStart', 'latEnd', 'lonEnd'])
allDevices.createOrReplaceTempView("alldevices")
allDevices.show(20, False)

In [ ]:
allDevices.count()

# Data visualization

In [ ]:
import pandas as pd
dfProcessed = allDevices.toPandas()

### Data check after transformations
We make an overview of the description of the data before and after the processing to check that it makes sense.

First we describe the data after processing. We observe allmost all the position data for start and end is identical, so just looking at the data we could tell that our car is always doing the same trip.

In [ ]:
dfProcessed.describe()

Then we also describe the data before doing any processing. Starting and end positions are mixed, so we can make much conclusions, but we can observe that the range of the data make sense.

In [ ]:
dfRaw = df.toPandas()
dfRaw.describe()

In [ ]:
dfProcessed = dfProcessed.sort_values("eventTimeStart")
dfProcessed

## Map visualization
When handling geospatial data, is a good practise to visualize it in a map to find insights or avoid erroneous assumptions on the data

In [ ]:
# A google maps Javascript API KEY is needed to see the maps in the notebook
import gmaps
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
startLocs = dfProcessed[['latStart','lonStart']]
endLocs = dfProcessed[['latEnd', 'lonEnd']]
center_coords = (32.920782, -96.950904)

### Starting positions visualization
A heatmap is displayed to observe the density of the trip-start positions in a map. We can see that there are three main areas around Dallas area were the cars start their trips.

In [ ]:
fig = gmaps.figure(center=center_coords, zoom_level=10)
heatmap = gmaps.heatmap_layer(startLocs)
heatmap.max_intensity = 2
heatmap.point_radius = 20
fig.add_layer(heatmap)
fig

### Ending positions visualization
As with starting positions, a heatmap is displayed to observe the density of the trip-end positions in a map. Again, there three main areas were the cars end their trips, showing that they likely do a go and return trip on the same areas.

In [ ]:
fig = gmaps.figure(center=center_coords, zoom_level=10)
heatmap = gmaps.heatmap_layer(endLocs)
heatmap.max_intensity = 2
heatmap.point_radius = 20
fig.add_layer(heatmap)
fig

Below, the positions markers combined are shown. trip-start are shown in green and trip-end in red

In [ ]:
center_coords = (32.874843, -96.947014)
figMarkers = gmaps.figure(center=center_coords, zoom_level=10)
trip_start_layer = gmaps.symbol_layer(
startLocs, fill_color="rgba(0, 150, 0, 0.4)",
stroke_color="rgba(0, 150, 0, 0.4)", scale=4
)
trip_end_layer = gmaps.symbol_layer(
endLocs, fill_color="rgba(200, 0, 0, 0.4)",
stroke_color="rgba(200, 0, 0, 0.4)", scale=4
)
#trip_start_layer = gmaps.symbol_layer(startLocs, fill_color="green", stroke_color="green", scale=3)
#trip_end_layer = gmaps.symbol_layer(endLocs, , stroke_color="red", scale=3, fill_color=None, fill_opacity=0.6)
figMarkers.add_layer(trip_start_layer)
figMarkers.add_layer(trip_end_layer)
figMarkers

After cleansing and processing we save the processed dataset to a csv file for its later use to build the prediction model.

In [ ]:
dfProcessed.to_csv('processed-dataset.csv')